In [ ]:
import streamlit as st
import openai
import pandas as pd

# Set up your Azure OpenAI endpoint and API key
OPENAI_API_KEY = "884039b2dd764c80a0297dfd9f57f6e2"
OPENAI_API_TYPE = "Azure"
AZURE_ENDPOINT = "https://demo-model.openai.azure.com/"
OPENAI_API_VERSION = "2024-02-01"
openai.api_type = OPENAI_API_TYPE
openai.api_base = AZURE_ENDPOINT
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY


# Load the mutual fund data
funds_data = pd.read_csv("./DataSets/MF_India_AI (1).csv")

def get_asset_allocation_report(age, annual_salary, monthly_expenses, monthly_loan, monthly_emergency_fund):
    prompt = f"""
    User Details:
    Age: {age}
    Annual Salary: ₹{annual_salary}
    Monthly Expenses: ₹{monthly_expenses}
    Monthly Loan Repayment: ₹{monthly_loan}
    Monthly Emergency Fund Contribution: ₹{monthly_emergency_fund}

    Calculate the monthly salary, monthly savings, determine the risk factor, and provide a recommended asset allocation strategy with the following breakdown:
    - Equity (High Risk)
    - Hybrid (Moderate Risk)
    - Debt (Low Risk)

    Provide the recommended allocation percentages and the exact investment amounts based on the monthly savings. Summarize the risk factor and the investment breakdown.
    
    Generate a report in the following template:
    1. Calculate Disposable Income:
    Annual Salary: ₹{annual_salary}
    Monthly Salary: 
    Monthly Expenses: ₹{monthly_expenses}
    Monthly Loan Repayment: ₹{monthly_loan}
    Monthly Emergency Fund Contribution: ₹{monthly_emergency_fund}
    Monthly Savings: 

    2. Risk Factor Determination:
    Age: {age}
    Risk Factor: 

    3. Asset Allocation Strategy:
    Equity (High Risk): 
    Hybrid (Moderate Risk): 
    Debt (Low Risk): 

    4. Recommended Allocation:
    Given your age and financial position, a higher allocation to equity is advisable.

    5. Investment Calculation:
    Let's allocate your monthly savings of ₹ into these categories:
    Equity: ₹
    Hybrid: ₹
    Debt: ₹

    6. Summary:
    Equity: % (₹ per month)
    Hybrid: % (₹ per month)
    Debt: % (₹ per month)
    Risk Factor: 
    """

    response = openai.Completion.create(
        engine="davinci",  # Replace with the deployment name you chose when you deployed the model
        prompt=prompt,
        max_tokens=500
    )

    return response.choices[0].text.strip()

def parse_report(raw_output):
    parsed_output = {}
    sections = raw_output.split('\n\n')

    for section in sections:
        lines = section.strip().split('\n')
        title = lines[0].strip()
        content = '\n'.join(lines[1:]).strip()
        parsed_output[title] = content

    return parsed_output

def extract_allocation_details(parsed_output):
    allocation = {'Equity': 0, 'Hybrid': 0, 'Debt': 0}
    for key, value in parsed_output.items():
        if key.startswith('Asset Allocation Strategy'):
            lines = value.split('\n')
            for line in lines:
                if 'Equity' in line:
                    allocation['Equity'] = float(line.split(':')[1].strip().replace('%', ''))
                elif 'Hybrid' in line:
                    allocation['Hybrid'] = float(line.split(':')[1].strip().replace('%', ''))
                elif 'Debt' in line:
                    allocation['Debt'] = float(line.split(':')[1].strip().replace('%', ''))
    return allocation

def recommend_funds(funds_data, allocation, monthly_savings):
    recommended_funds = {}
    for category in allocation:
        percentage = allocation[category]
        amount = (percentage / 100) * monthly_savings
        funds = funds_data[funds_data['Category'] == category].nlargest(3, 'Returns')  # Top 3 funds based on Returns
        recommended_funds[category] = {'amount': amount, 'funds': funds}
    return recommended_funds

def display_recommended_funds(recommended_funds):
    for category, details in recommended_funds.items():
        st.subheader(f'{category} Funds')
        st.write(f'Allocated Amount: ₹{details["amount"]:.2f}')
        st.table(details['funds'])

# Streamlit app
st.title('Asset Allocation Report Generator')

age = st.number_input('Enter your age:', min_value=1, max_value=100, value=21)
annual_salary = st.number_input('Enter your annual salary (₹):', min_value=0, value=1000000)
monthly_expenses = st.number_input('Enter your monthly expenses (₹):', min_value=0, value=50000)
monthly_loan = st.number_input('Enter your monthly loan repayment (₹):', min_value=0, value=10000)
monthly_emergency_fund = st.number_input('Enter your monthly emergency fund contribution (₹):', min_value=0, value=10000)

if st.button('Generate Report'):
    raw_report = get_asset_allocation_report(
        age=age,
        annual_salary=annual_salary,
        monthly_expenses=monthly_expenses,
        monthly_loan=monthly_loan,
        monthly_emergency_fund=monthly_emergency_fund
    )

    parsed_report = parse_report(raw_report)
    st.subheader("Generated Report")
    display_report(parsed_report)
    
    # Extracting monthly savings from the parsed report
    monthly_savings_line = [line for line in parsed_report['1. Calculate Disposable Income:'].split('\n') if 'Monthly Savings' in line][0]
    monthly_savings = float(monthly_savings_line.split(':')[1].strip().replace('₹', '').replace(',', ''))
    
    # Extracting allocation details
    allocation = extract_allocation_details(parsed_report)
    
    # Recommending funds based on the allocation and savings
    recommended_funds = recommend_funds(funds_data, allocation, monthly_savings)
    display_recommended_funds(recommended_funds)
